In [1]:
using Revise, Pkg
Pkg.activate(".")

In [22]:
using Napari, ThreeDeconv, FFTW, Images, Printf, Statistics, HDF5, CUDA

In [9]:
img = Float32.(load("data.tif"));
psf = Float32.(load("psf.tif"));
img ./= maximum(img);

In [27]:
γ, σ = ThreeDeconv.noise_estimation(img, maxnum_pairs = 200)
@printf "Gain: %.1f, Read noise std.: %.1f \n" γ σ


options =
    ThreeDeconv.DeconvolutionOptions(max_iters = 300, show_trace = true, check_every = 50)
reg = 1e-7

Computing local noise variance.
Initializing parameters by least-squares.
Initialization done.
Starting likelihood maximization.
Finished the maximization.
Estimated noise parameters are a = 0.001, b = 0.000.
Estimated parameters for Poisson-Gaussian noise model
γ: 0.0011922871862607468, σ: 0.008916181746551226
Gain: 0.0, Read noise std.: 0.0 


1.0e-7

In [11]:
using ThreeDeconv

In [29]:
ThreeDeconv.to_gpu_or_not_to_gpu(randn((2,2)))

2×2 CuArray{Float64, 2}:
  1.17254  -0.871916
 -2.27469  -0.319056

In [32]:
CUDA.@time result = ThreeDeconv.deconvolve(img, psf, γ, σ, reg, options = options)

No clipping performed.
Iter     rel_diff      time  
------   ---------   --------
     0         NaN       0.01
    50      0.0183       4.59
   100     0.00897       9.03
   150     0.00556      13.47
   200     0.00406      17.91
   250     0.00314      22.36
   300     0.00259      27.09
 27.892288 seconds (714.41 k CPU allocations: 871.073 MiB, 5.26% gc time) (25.26 k GPU allocations: 1.546 TiB, 5.31% gc time of which 13.00% spent allocating)


ThreeDeconv.OptimizationResult(Float32[63.767864 53.959984 … -0.13652916 4.9848537; 137.44115 167.44516 … 0.48610437 34.149643; … ; 0.15088415 -0.28683376 … 0.17388922 -0.78955245; 10.2367325 0.7265222 … -0.35017133 0.7125435]

Float32[14.730013 11.864308 … 0.47252095 0.8695887; 74.45057 88.77038 … 0.15573338 25.253944; … ; 0.13266659 0.19533825 … -0.09922834 -0.19094405; -0.124617964 -0.2276414 … -0.63622195 -0.24363491]

Float32[-0.43170547 -0.10282111 … 0.34445179 -0.38331664; 3.4263556 -2.153527 … -1.1890757 7.103594; … ; -0.3080883 0.09846687 … -0.077198856 0.20547056; 0.40997142 0.06541221 … -0.14634892 -0.5953701]

...

Float32[13.4478 -0.45761764 … -0.2881111 0.60774136; 20.169199 5.2130947 … -0.8893171 0.6526987; … ; -0.8908521 0.08622739 … -0.00044098496 -0.42870918; -1.6711092 1.1891503 … -0.47586724 -0.018650347]

Float32[62.42531 25.207197 … -0.015871823 10.506632; 83.77694 78.11122 … -0.13785528 10.502655; … ; -0.031119477 0.30504316 … 0.45683423 0.26854968; 22.616898 -0.

In [18]:
h5write("/home/fxw/Nextcloud/Uni/DeconvOptim.jl/good_data/ThreeDeconv.h5", "res", result.x)